In [106]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from string import punctuation
custom = stop_words+list(punctuation)
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
def my_tokenizer(s):
    try:
        s = s.lower() # downcase
    except:
        s = str(s).lower()
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in custom] # remove stopwords
    tokens = [t for t in tokens if not any(c.isdigit() for c in t)] # remove any digits, i.e. "3rd edition"
    return tokens

def clean_text(df, column_1, column_2):
    df = df[[column_1, column_2]]
    text = df[column_1].tolist()
    cleaned_text = []
    for x in text:
        cleaned_text.append(my_tokenizer(x))
    joined_text = []
    for x in cleaned_text:
        joined_text.append(' '.join(x))
    df1 = pd.DataFrame({'Message':joined_text})
    df1["sentiment"] = pd.Series(df[column_2].tolist())
    df1["sentiment"] = df1["sentiment"].str.lower()
    return df1

[nltk_data] Downloading package stopwords to /home/nbuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/nbuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/nbuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [107]:
df_jan = pd.read_excel("output_jan.xlsx")
df_feb = pd.read_excel("output_feb.xlsx")
df_mar = pd.read_excel("output_mar.xlsx")
df_apr = pd.read_csv("output_apr.txt")
df_may = pd.read_csv("output_may.txt")
df_jun = pd.read_csv("output_june.txt")
df_jan_cleaned = clean_text(df_jan, "Message", "Revsied")
df_feb_cleaned = clean_text(df_feb, "Message", "revised")
df_mar_cleaned = clean_text(df_mar, "Message", "Revised")
df_apr_cleaned = clean_text(df_apr, "Message", "revised")
df_may_cleaned = clean_text(df_may, "Message", "Revised")
df_jun_cleaned = clean_text(df_jun, "Message", "Revised")

In [108]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words="english")
le = LabelEncoder()
from sklearn.metrics import accuracy_score,roc_auc_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split as tts
from sklearn.naive_bayes import MultinomialNB
!pip install imblearn
from imblearn.over_sampling import SMOTE
def vectorize_and_encode(df):
    df = df.dropna()
    tfidf.fit(df["Message"])
    vector = tfidf.transform(df["Message"])
    vector_values_array = vector.toarray()
    X = vector_values_array
    y = le.fit_transform(df["sentiment"])
    return X, y

In [109]:
df = pd.concat([df_jan_cleaned, df_feb_cleaned, df_mar_cleaned, df_apr_cleaned, df_jun_cleaned])

In [110]:
# X,y = vectorize_and_encode(df)

In [111]:
# X.shape

In [112]:
df.shape

(7021, 2)

In [113]:
# def smote(X,y):
#     sm = SMOTE(random_state=42)
#     X_res, y_res = sm.fit_sample(X, y.ravel())
#     return X,y

In [114]:
# X,y = smote(X,y)

In [115]:
# X.shape

In [116]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    text = str(text)
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [117]:
def test(csv_file):
    try:
        test = pd.read_csv(csv_file)
    except:
        test = pd.read_csv(csv_file, encoding="Latin-1")
    message_work_msg_cleaned =[]
    for x in range(test.shape[0]):
        message_work_msg_cleaned.append(tweet_cleaner(test["content"].iloc[x]))    
    message_work_sentiment = test["sentiment"].tolist()
    df1 = pd.DataFrame({'Message':message_work_msg_cleaned})
    df1["sentiment"] = pd.Series(message_work_sentiment)
    df1["sentiment"] = df1["sentiment"].str.lower()
    return df1
    

In [118]:
# message_work_msg_cleaned =[]
# for x in range(test.shape[0]):
#     message_work_msg_cleaned.append(tweet_cleaner(test["content"].iloc[x]))    

In [119]:
# len(message_work_msg_cleaned)

In [120]:
# message_work_sentiment = test["sentiment"].tolist()

In [121]:
# test = test("unilever ml.csv")

In [122]:
# test.head()

In [123]:
# test_cleaned = clean_text(test, "Message", "sentiment")
test_cleaned = df_may_cleaned.copy()

In [124]:
test_cleaned.shape

(1923, 2)

In [125]:
df.shape

(7021, 2)

In [126]:
# test_cleaned = df_may_cleaned.copy()

In [127]:
test_cleaned.shape

(1923, 2)

In [128]:
test_cleaned = test_cleaned.dropna()
test_cleaned.isnull().sum()
df = df.dropna()
df.isnull().sum()
print (test_cleaned.shape)
print (df.shape)

(1923, 2)
(6819, 2)


In [129]:
8742+483
# 1923+6819



9225

In [130]:
def vectorize_test(test):
    df1 = pd.concat([df, test])
    tfidf.fit(df1["Message"])
    vector = tfidf.transform(df1["Message"])
    vector_values_array = vector.toarray()
    X = vector_values_array
    y = le.fit_transform(df["sentiment"])
    return X, y

In [131]:
X,y = vectorize_test(test_cleaned)

In [132]:
X.shape

(8742, 8705)

In [133]:
y.shape

(6819,)

In [134]:
X_train = X[:y.shape[0]]
y_train = y
X_test = X[y.shape[0]:]

In [135]:
y_test = le.fit_transform(test_cleaned["sentiment"])

In [136]:
y_test.shape

(1923,)

In [137]:
# sm = SMOTE(random_state=42)
# X_train, y_train = sm.fit_sample(X_train, y_train.ravel())

# nb = MultinomialNB()

# nb.fit(X_train,y_train)
# y_pred = nb.predict(X_test)
# print (accuracy_score(y_test,y_pred))
# print (classification_report(y_test,y_pred))
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_sample(X_train, y_train.ravel())
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)
print (accuracy_score(y_test,y_pred))
print (classification_report(y_test,y_pred))

0.7914716588663546
              precision    recall  f1-score   support

           0       0.58      0.66      0.62       312
           1       0.85      0.86      0.85      1243
           2       0.81      0.68      0.74       368

   micro avg       0.79      0.79      0.79      1923
   macro avg       0.75      0.73      0.74      1923
weighted avg       0.80      0.79      0.79      1923

